In [1]:
#data input, output
import os
import sys
#split only ㅋ
import re
# 뭐에 쓰드라...
import math
# data 전처리, 분석용
import pandas as pd
import numpy as np
import scipy as sp
from scipy import stats as st
import statsmodels.stats.api as sms
# 서버 requests
import requests
from requests.auth import HTTPBasicAuth
# age 추출
import datetime
from datetime import date
from dateutil.parser import parse
# figure 작업 용
import matplotlib as mpl
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

#이미지 작업용
import matplotlib.image as mpimg # np.array로 이미지 로드
from PIL import Image # 오브젝트로 로드  -> np.array로 캐스팅

In [2]:
def figure_out_perf(s_data = {}, gtype = 'bar', level = False):
    """
        작성자: sjyoo
        작성일: 190704
        기능: 행동 데이터 figure 추출
        입력: s_data(score 분리 데이터)
        출력: figure(A to Z).png export
        용례: figure_out_perf(s_data, gtype = ['bar', 'stem', 'image 작성중..', 'dist 따로 만들듯...'], 
                                level = boolen(지금은 레벨 점수 적용 여부))
    """
    #폰트 설정
    sns.set(font_scale=0.5)
    mpl.rcParams['axes.unicode_minus'] = False
    path = 'C:/Windows/Fonts/HYSNRL.ttf'
    font_name = fm.FontProperties(fname=path).get_name()
    plt.rc('font', family=font_name)
    
    for task in s_data:
        if 'REST' in task or 'VFT' in task:
            pass
        else:
            s_list = s_data[task].childName.unique()

            for child in s_list:
                s_child = s_data[task][s_data[task].childName == child]
                s_child = s_child.dropna(how = 'any')
                y = s_child['%s_Cor' % task].astype('int')
                y = y.replace(0, -1)
                
                #레벨 적용
                if level == True:
                    LV = [int(lv[2:]) for lv in s_child['%s_LV' % task]]
                    y = np.multiply(LV, y)
                
                fig = plt.figure()
                ax =plt.subplot(111)
                x_name=  s_child['ctime']
                n_groups = len(x_name)
                index = np.arange(n_groups)
                bar_width = 0.35
                opacity = 1
                
                # bar graph
                if gtype == 'bar':
                   

                    ax.bar(index, y.replace(y[y<0], np.nan), alpha=opacity, color='b')
                    ax.bar(index, y.replace(y[y>0], np.nan), alpha=opacity, color='r')
                    ax.set_xlabel('time')
                    ax.set_ylabel('%s_Cor' % task)
                    ax.set_title('%s Bar Chart' % (task))
#                     ax.set_ylim(-3,3)
                    
                    plt.savefig('%s_%s_%s.png' % (child, task, gtype), format = 'png', dpi = 300)
                    plt.show()

                # stem graph    
                elif gtype == 'stem':

                    ax.stem(index, y.replace(y[y<0], np.nan), markerfmt = 'C3o', basefmt = 'r', linefmt='grey')
                    ax.stem(index, y.replacey(y[y>0], np.nan), markerfmt = 'C0o', basefmt= 'r', linefmt='grey' )
                    ax.set_xlabel('time')
                    ax.set_ylabel('%s_Cor' % task)
                    ax.set_title('%s Stem Chart' % (task))
#                     ax.set_ylim(-3,3)
                    
                    plt.savefig('%s_%s_%s.png' % (child, task, gtype), format = 'png', dpi = 300)
                    plt.show()

                # Distribution figure
                elif gtype == 'image':
                    path = os.getcwd()
                    imf = os.path.join(path, 'graphboard.png')

                    image = mpimg.imread(imf)
                    plt.imshow(image)
#                     height, width, layer = image.shape
#                     f, axes = plt.subplots(2, 2, figsize=(8, 8*height/width))
                    ## original img plotting 
#                     axes[0][0].imshow(image[:, :, :]), axes[0][0].axis('off')
#                     axes[0][0].set_xticks([]), axes[0][0].set_yticks([])# 이걸 하지 않으면 tick이 남아있어서 간격이 생김. 
                    # Red, Green, Blue로 구분하여 표현. colormap 또한, 그 형식에 맞춰서 표현 
                    # 실제 그림을 보면 색깔별로 어느 정도 구분되어 있는 것을 알 수 있음. 
#                     cmaps = [plt.cm.Reds, plt.cm.Greens, plt.cm.Blues]
#                     for i in range(1, 4):
#                         axes[i//2][i%2].imshow(image[:, :, i-1], cmap=cmaps[i-1])
#                         axes[i//2][i%2].set_xticks([]), axes[i//2][i%2].set_yticks([])# 이걸 하지 않으면 tick이 남아있어서 간격이 생김. 
#                         axes[i//2][i%2].axis('off')
#                     plt.subplots_adjust(left = 0, bottom = 0, right = 1, top = 1, hspace = 0, wspace = 0)
                    ## sutplots_adjust는 subplot 간에 간격을 붙이려고 쓴건데, 쓰고보니 어떻게 쓰는건지 모르겠음. 그냥 모르겠음...
                    plt.margins(0, 0, tight=False)
                    # pad_inches를 0으로 두고 저장하면, 공백없이 저장됨. 
#                     plt.savefig("../../assets/images/markdown_img/180628_1935_google_rgb.svg", pad_inches=0)
                    plt.show()
                    
                elif gtype == 'dist':

                    a = List[feature].dropna()
                    low = a[a < np.percentile(a, 30)].count()
                    high = a[a > np.percentile(a, 70)].count()
                    middle = a[(a <= np.percentile(a, 70))&(a >= np.percentile(a, 30))].count()

                    points = sns.distplot(a.dropna() 
            #                               , bins = 1000
                                          , fit=sp.stats.norm
                                          , hist =False, kde = True, kde_kws = {'shade' :True, 'kernel' : 'gau'
            #                                                                  , 'bw' : 0.1
                                                                                , 'clip' :(a.min(),a.max()) 
                                                                            }
                                         ).get_lines()[0].get_data()

                    x = points[0]
                    y = points[1]
            #         th = sms.DescrStatsW(a).tconfint_mean(0.0001)[0]
            #         se = sms.DescrStatsW(a).tconfint_mean(0.0001)[1]
                    plt.fill_between(x,y, where = x > np.percentile(a, 75), color='y', alpha=0.7, label = '높음')
                    plt.fill_between(x,y, where = x <  np.percentile(a, 25), color='y', alpha=0.7, label = '낮음')
                    plt.fill_between(x,y, where = (x <=  np.percentile(a, 75)) & (x >= np.percentile(a, 25)), color='b', alpha=0.7, label = '보통')
                    plt.fill_between(x,y, where = (x > np.percentile(a, 90)) &  (x < np.percentile(a, 10)), color='r', alpha=0.7, label = '높음')

            #         hito = plt.hist(a, alpha = 0.5, align  = 'right' )
            #         plt.text(.50, .5, '낮음 = %s , 중간 = %s , 보통 = %s' % (low, middle, high), horizontalalignment='center',
            #      verticalalignment='center',
            #      textcoords = 'axes fraction')
            #         plt.text(x.min(), hito[0].max(), '낮음 = %s , 보통 = %s, 높음 = %s' % (low, middle, high), style='italic',
            #         bbox={'facecolor':'red', 'alpha':0.5, 'pad':10})
            #         plt.annotate('낮음 = %s' % low, xy = (np.percentile(a, 25),np.percentile(a, 25)), xytext=(0.25, 0.5), textcoords = 'axes fraction', arrowprops = dict(color = 'g', alpha= 0.5))
            #         plt.annotate('중간 = %s' % middle, xy = (np.percentile(a, 50),np.percentile(a, 50)), xytext=(0.5, 0.7), textcoords = 'axes fraction', arrowprops = dict(color = 'g', alpha= 0.5))
            #         plt.annotate('높음 = %s' % high, xy = (np.percentile(a, 75),np.percentile(a, 75)), xytext=(0.75, 0.5), textcoords = 'axes fraction', arrowprops = dict(color = 'g', alpha= 0.5))


                    plt.savefig('%s_%s.png' % (feature, gtype), format = 'png', dpi = 1200)
                    plt.show()
    return

def data_scoring(raw = pd.DataFrame(), export = 'f_data'):
    """
        작성자: sjyoo
        작성일: 190704
        기능: 각 행동 데이터 response, index, ctime, taskUUID를 분리하여 코딩
        입력: raw = pd.DataFrame(raw data)
        출력: f_data.xlsx export & figure 용 data
        용례: s_data = data_scoring(list_raw, data, str(export excel file name))
    """
    tasks = {1:'REST',2:'CBTTF',3:'CBTTB',4:'GNG',5:'TWOBACK',6:'STRC',7:'STRI',8:'VFT',9:'REPEAT',10:'MIND',11:'FOLD'}
    GNG_LV = {'LV1':2500, 'LV2': 2250, 'LV3': 2000, 'LV4': 1750, 'LV5': 1500, 'LV6': 1250, 'LV7': 1000, 'LV8': 750, 'LV9': 750, 'LV10': 750, 'LV11': 750, 'LV12': 750} 
    print('scoring data listing 시작')
    s_data = {}
    with pd.ExcelWriter('C:/Users/OBELAB_JH_DESKTOP/Documents/GitHub/testfornothing/OBE/data/'+ export + '.xlsx') as writer:
        for k, v in tasks.items():
            print('key : '+ str(k) + ', value : ' + v)
            s_data[v] = raw[raw['taskType'] == str(k)]
            s_data[v] = s_data[v].replace(['TASKMARKER','OUTTOUCH', 'TASKMARKER\r','OUTTOUCH\r'],[np.nan, np.nan, np.nan, np.nan]).dropna(axis = 1, how = 'all')
#             s_data[v].fillna('')
            if 'GNG' in v or 'STRC' in v or 'STRI' in v or 'TWOBACK' in v or 'CBTTF' in v or 'CBTTB' in v or 'MIND' in v or 'REPEAT' in v or 'FOLD' in v:
                LV, Cor, Act, RT = [], [], [], []
                for ra in s_data[v]['ra']:
                    if pd.isnull(ra) == False:
                        marker = ra.split('/')
                    else:
                        marker = ['LV', 'Cor', 'Act', 'RT']
                        
#                     print[marker]    
                    LV.append(marker[0])
                    Cor.append(marker[1])
                    Act.append(marker[2])
                    RT.append(marker[3])
                s_data[v]['%s_LV' % v] = LV
                s_data[v]['%s_Cor' % v] = Cor
                s_data[v]['%s_Act' % v] = Act
                s_data[v]['%s_RT' % v] = RT
                print('%s 과제 완료' % v)
            else:
                pass

            s_data[v].to_excel(writer, sheet_name=v, index = None)
            
    print('정리 완료')
    return s_data


def data_merge(List = pd.DataFrame(), raw_data = {}, export = 'merge'):
    """
        작성자: sjyoo
        작성일: 190612
        기능: list와 raw 파일 COR, ACC, RT 등을 merge
        입력: list = pd.DataFrame(), raw_data = dict{}
        출력: merge.xlsx export
        용례: data_merge(list_raw, raw_data, str(export excel file name))
    """
    tasks = {1:'REST',2:'CBTTF',3:'CBTTB',4:'GNG',5:'TWOBACK',6:'STRC',7:'STRI',8:'VFT',9:'REPEAT',10:'MIND',11:'FOLD'}
    GNG_LV = {'LV1':2500, 'LV2': 2250, 'LV3': 2000, 'LV4': 1750, 'LV5': 1500, 'LV6': 1250, 'LV7': 1000, 'LV8': 750, 'LV9': 750, 'LV10': 750, 'LV11': 750, 'LV12': 750} 
#     task_stage = {'GNG':,
#                  'CBTTF':,
#                  'CBTTB':, 
#                  'REPEAT':,
#                  'FOLD': }
    print('merge 시작')
    m_data = {}
    m_data = raw_data
    for k, v in tasks.items():
        print('key : '+ str(k) + ', value : ' + v)
        m_data[v] = m_data[v].replace(['TASKMARKER','OUTTOUCH', 'TASKMARKER\r','OUTTOUCH\r'],[np.nan, np.nan, np.nan, np.nan]).T.dropna(axis = 1, how = 'all')
#         m_data[v].fillna('')
        pack = pd.DataFrame()
        idx = m_data[v].index
        col = m_data[v].columns
        pack['taskUUID'] = idx
        RT, Acc, Max, Cor_list = [], [], [], []
        if 'GNG' in v:
            for i in idx:
                Cor, LV = 0, 0
                RT_list, LV_list = [], []
                for j in col:
                    if pd.isnull(m_data[v][j][i]) == False:
                        marker = m_data[v][j][i].split('/')
                        if 'LV' in marker[0]:
                            Cor += int(marker[1])
                            LV += 1
                            LV_list.append(int(marker[0][2:]))
                            if '1-1' in marker[2]:
                                if 'LV' in marker[3]:
                                    pass
                                else:
                                    if [int(marker[3]) < t for l, t in GNG_LV.items() if marker[0] == l]:
                                        RTint = int(marker[3])
                                        RT_list.append(RTint)
                        else:
                            pass
                    else:
                        pass
                if not LV_list:
                    Max.append(1)
                else:
                    Max.append(max(LV_list))
                Cor_list.append(Cor)
                if len(RT_list) is not 0:
                    RT.append(sum(RT_list) / len(RT_list))
                else:
                    RT.append(0)
                
                if LV == 0:
                    Acc.append(0)
                else:
                    Acc.append(Cor/LV)
                
            pack['%s_Cor' % v] = Cor_list
            pack['%s_RT' % v] = RT
            pack['%s_ACC' % v] = Acc
            pack['%s_max' % v] = Max
            print('정답률, 반응시간')
            print('과제 완료')
        
        elif 'STRC' in v or 'STRI' in v:
            for i in idx:
                Cor, LV = 0, 0
                RT_list = []
                for j in col:
                    if pd.isnull(m_data[v][j][i]) == False:
                        marker = m_data[v][j][i].split('/')
                        if 'LV' in marker[0]:
                            Cor += int(marker[1])
                            LV += 1
                            if marker[1] == '1':
                                if 'LV' in marker[3]:
                                    pass
                                else:
                                    if int(marker[3]) < 2000 :
                                        RTint = int(marker[3])
                                        RT_list.append(RTint)
                        else:
                            pass
                    else:
                        pass
                Cor_list.append(Cor)
                if len(RT_list) is not 0:
                    RT.append(sum(RT_list) / len(RT_list))
                else:
                    RT.append(0)
                
                if LV == 0:
                    Acc.append(0)
                else:
                    Acc.append(Cor/LV)
                
            pack['%s_Cor' % v] = Cor_list
            pack['%s_RT' % v] = RT
            pack['%s_ACC' % v] = Acc
            print('정답률, 반응시간')
            print('과제 완료')
            
        elif 'TWOBACK' in v:
            for i in idx:
                Cor, LV = 0, 0
                RT_list = []
                for j in col:
                    if pd.isnull(m_data[v][j][i]) == False:
                        marker = m_data[v][j][i].split('/')
                        if 'LV' in marker[0]:
                            Cor += int(marker[1])
                            LV += 1
                            if '1-1' in marker[2]:
                                if 'LV' in marker[3]:
                                    pass
                                else:
#                                 if int(marker[3]) < 2000 :
                                    RTint = int(marker[3])
                                    RT_list.append(RTint)
                        else:
                            pass
                    else:
                        pass
                Cor_list.append(Cor)
                if len(RT_list) is not 0:
                    RT.append(sum(RT_list) / len(RT_list))
                else:
                    RT.append(0)
                
                if LV == 0:
                    Acc.append(0)
                else:
                    Acc.append(Cor/LV)
                
            pack['%s_Cor' % v] = Cor_list
            pack['%s_RT' % v] = RT
            pack['%s_ACC' % v] = Acc
            print('정답률, 반응시간')
            print('과제 완료')
            
            
        elif 'CBTTF' in v or 'CBTTB' in v:
            for i in idx:
                Cor, LV = 0, 0
                LV_list = []
                for j in col:
                    if pd.isnull(m_data[v][j][i]) == False:
                        marker = m_data[v][j][i].split('/')
                        if 'LV' in marker[0]:
                            Cor += int(marker[1])
                            LV += 1
                            LV_list.append(int(marker[0][2:]))
                        else:
                            pass
                    else:
                        pass
#                 print(LV_list)
                if not LV_list:
                    Max.append(1)
                else:
                    Max.append(max(LV_list))
                Cor_list.append(Cor)
                if LV == 0:
                    Acc.append(0)
                else:
                    Acc.append(Cor/LV)
            pack['%s_Cor' % v] = Cor_list
            pack['%s_ACC' % v] = Acc
            pack['%s_max' % v] = Max
            print('정답률')
            print('과제 완료')
            
        elif 'MIND' in v or 'REPEAT' in v or 'FOLD' in v:
            for i in idx:
                Cor, LV = 0, 0
                LV_list = []
                for j in col:
                    if pd.isnull(m_data[v][j][i]) == False:
                        marker = m_data[v][j][i].split('/')
                        if 'LV' in marker[0]:
                            Cor += int(marker[1])
                            LV += 1
                            LV_list.append(int(marker[0][2:]))
                        else:
                            pass
                    else:
                        pass
#                 print(LV_list)
                if not LV_list:
                    Max.append(1)
                else:
                    Max.append(max(LV_list))
                Cor_list.append(Cor)
                if LV == 0:
                    Acc.append(0)
                else:
                    Acc.append(Cor/LV)
            pack['%s_Cor' % v] = Cor_list
            pack['%s_ACC' % v] = Acc
            pack['%s_max' % v] = Max
            print('정답률')
            print('과제 완료')    
        else:
            print('pass')
            pass
        
        List = List.merge(pack, on='taskUUID', how = 'outer')
    List['GNG_z'] = List['GNG_ACC'] / List['GNG_RT']*1000
    List['STRC_z'] =  List['STRI_ACC'] / List['STRI_RT']*1000
    List['STRI_z'] =  List['STRC_ACC'] / List['STRC_RT']*1000
    List['STRE'] =  (List['STRC_RT'] - List['STRI_RT'])/1000
    List['STRE_z'] =  List['STRI_z'] - List['STRC_z']
    List.to_excel(export + '.xlsx', sheet_name = 'data')
    print('caculate 완료')
    return List
    
def raw_to_data(raw):
    """
        작성자: sjyoo
        작성일: 190611
        기능: table을 분석용 dict로 내보내기
        입력: raw table(dataframe)
        출력: raw_data = dict{pd.DataFrame{task pivot)}
        용례: raw_data = raw_to_data(pandas DataFrame(raw))
        메모: 데이터에 따라 pivot/pivot_table 조절
    """
    tasks = {1:'REST',2:'CBTTF',3:'CBTTB',4:'GNG',5:'TWOBACK',6:'STRC',7:'STRI',8:'VFT',9:'REPEAT',10:'MIND',11:'FOLD'}
    task_num = raw['taskType'].unique()
    raw_data = {}
    for task in tasks.values():
        raw_data[task]= pd.DataFrame()
#     print(raw)
    for k in task_num:
        task_ind = raw.query('taskType == "' + k +'"')
#         print(task_ind)
        # data = task_ind.pivot(columns = 'childName', values = 'raw')
        # data = pd.pivot_table(task_ind, index = 'index', columns = 'childName', values = 'raw', aggfunc=np.sum, margins = True)
        raw_data[tasks[int(k)]] = pd.pivot_table(task_ind, index = 'index', columns = 'taskUUID', values = 'ra', aggfunc=np.sum)
        print('%s  실행' % k)
    return raw_data

def request_to_table(url, uuid, my_id, my_pw):
    """
        작성자: cmlee
        수정자: sjyoo
        작성일: 190509
        수정일: 190611 
        기능: 요청 정보를 이용해서 서버에서 데이터 테이블 형태로 반환
        입력: url, id, pw, taskuuid
        출력: pandas DataFrame : header + contents
        용례: raw, list_raw = request_to_table(str(api_url), str(uuid)/ list(uuid), str(my_id), str(my_pw))
    """
    raw = pd.DataFrame()
#     list_raw = pd.DataFrame()
    print('table 요청 시작')
    for child in uuid:
        print(child + '_로딩 중....')
        url_uuid = url + '?taskUUID=' + child
        r = requests.get(url=url_uuid, auth=HTTPBasicAuth(my_id, my_pw))
        r = r.content.decode("utf-8")
        r = re.split('\n|\r', r)
        t_head = r.pop(0).split(',')
        t_head[-1] = t_head[-1][0:-1]
        t_body = [t.split(',') for t in r if t != '']
        full = pd.DataFrame(t_body, columns=t_head)
        full = full[full.taskUUID != '']
        tasks = full['taskType'].unique()
        for task in tasks:
            task_raw = full.query('taskType == "' + str(task) +'"')
            mx = task_raw.taskSubID.max()
            if mx != '':
                t_mx_raw = task_raw.query('taskSubID == "'+str(mx)+'"')
                raw = pd.concat([raw, t_mx_raw], axis=0)
        print('%s 완료' % child)
#     list_raw['name'] = raw['childName']
#     list_raw['uuid'] = raw['taskUUID']
#     list_raw = list_raw.drop_duplicates(keep = 'last')
    print("raw export 완료")
#     return raw, list_raw
    return raw


def access_to_table(connect_info, uuid, table = 'uuid'):
    """
        작성자: sjyoo
        작성일: 190611
        기능: table 접속
        입력: connect_info: id, pw; uuid = taskuuid; table = 받고 싶은 데이터 table
        출력: pandas DataFrame : 현재는 performance 
        용례: raw = access_to_table(connect_info, uuid, table = 'uuid'&'raw'&'perf')
    """
    # base_url = 'http://ec2-15-164-48-95.ap-northeast-2.compute.amazonaws.com'
    base_url = 'http://obelab-api.com'
    api_uuid = base_url + '/api/analysis/taskuuid/'
    api_raw = base_url + '/api/analysis/metaraw/'
    api_perf = base_url + '/api/analysis/metamarker/'
    print('table access 시작')
    if table == 'uuid':
        return request_to_table(api_uuid, uuid, connect_info[0], connect_info[1])
    elif table == 'raw':
        return request_to_table(api_raw, uuid, connect_info[0], connect_info[1])
    elif table == 'perf':
        return request_to_table(api_perf, uuid, connect_info[0], connect_info[1])
    print("raw 완료")


def get_UUID(connect_info, date1 = '2000-01-01', date2 = '2030-01-01'):
    """
        작성자: sjyoo
        작성일: 190612
        기능: date로 아이디 뽑아내기. UUID 뽑을 거임.
        입력: 접속 정보, 시작일, 종료일
        출력: uuid 리스트
        용례: get_UUID(접속정보, 시작일, 종료일)
    """
    base_url = 'http://obelab-api.com'
    api_uuid = base_url + '/api/analysis/taskuuid/'
    uuid = pd.DataFrame()
    #지금은 다 불러와서 date로 filtering
    r = requests.get(url=api_uuid, auth=HTTPBasicAuth(connect_info[0], connect_info[1]))
    r = r.content.decode("utf-8")
    r = re.split('\n|\r', r)
    t_head = r.pop(0).split(',')
    t_head[-1] = t_head[-1][0:-1]
    t_body = [t.split(',') for t in r]
    data = pd.DataFrame(t_body, columns=t_head)
    data = data.query('taskCreated >= "' + date1 + '" and taskCreated <= "' + date2 +'"')
    pack = []
    for b, c in zip(data.childBirth, data.taskCreated):
#         birth = datetime.datetime.strptime(b, '%Y/%m/%d')
        birth = parse(b)
        testdate = datetime.datetime.strptime(c, '%Y-%m-%d %H:%M:%S.%f+00:00')
        ag = testdate.year - birth.year
        pack.append(ag)
    data['age'] = pack
    data.pop('sn')
    data = data.drop_duplicates(keep = 'last')
    return data


def server_connect_info():
    my_id = 'admin'
    my_pw = 'obe1234'

    return my_id, my_pw

In [3]:
connect_info = server_connect_info()
list_raw = get_UUID(connect_info, '2019-04-01', '2019-07-01')
target_child = list_raw.taskUUID
# target_child = ['HgwQ9iNB', 'vnTJNL39']
# print(target_child)

C:\Users\OBELAB_JH_DESKTOP\Anaconda3\lib\site-packages\ipykernel_launcher.py:522: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
# raw, list_raw = access_to_table(connect_info = connect_info, uuid=target_child, table = 'perf') #한예준
raw = access_to_table(connect_info = connect_info, uuid=target_child, table = 'perf') #한예준

table access 시작
table 요청 시작
gC9AsvWo_로딩 중....
gC9AsvWo 완료
kNkYISVX_로딩 중....
kNkYISVX 완료
XWs8g7W7_로딩 중....
XWs8g7W7 완료
0VbY8s8y_로딩 중....
0VbY8s8y 완료
4amBUpGr_로딩 중....
4amBUpGr 완료
ltaugEV0_로딩 중....
ltaugEV0 완료
DSy6ICtC_로딩 중....
DSy6ICtC 완료
2fJ6f7jV_로딩 중....
2fJ6f7jV 완료
BB9c6XWE_로딩 중....
BB9c6XWE 완료
bLClV9FD_로딩 중....
bLClV9FD 완료
VgV9xVTl_로딩 중....
VgV9xVTl 완료
rFBukveE_로딩 중....
rFBukveE 완료
Oa6gbbGg_로딩 중....
Oa6gbbGg 완료
SmRgdCu8_로딩 중....
SmRgdCu8 완료
DaBUUQI1_로딩 중....
DaBUUQI1 완료
tKGlt6hI_로딩 중....
tKGlt6hI 완료
RiWmyDhW_로딩 중....
RiWmyDhW 완료
xU1X7Dl9_로딩 중....
xU1X7Dl9 완료
x7LKQk4s_로딩 중....
x7LKQk4s 완료
AYmcwoid_로딩 중....
AYmcwoid 완료
RdE7k5PY_로딩 중....
RdE7k5PY 완료
u02vI3so_로딩 중....
u02vI3so 완료
Q5E07Fpg_로딩 중....
Q5E07Fpg 완료
wgYUFVYC_로딩 중....
wgYUFVYC 완료
5Kezxygb_로딩 중....
5Kezxygb 완료
uEl2EHo1_로딩 중....
uEl2EHo1 완료
MKfcvdHK_로딩 중....
MKfcvdHK 완료
RnCJh4Nq_로딩 중....
RnCJh4Nq 완료
k6BxPKL3_로딩 중....
k6BxPKL3 완료
yvQ4lcXD_로딩 중....
yvQ4lcXD 완료
ER1862nl_로딩 중....
ER1862nl 완료
OxWGYVDi_로딩 중....
OxWGYVDi 완료
eMxlSIcI_로딩 

In [5]:
raw_data = raw_to_data(raw)

1  실행
2  실행
3  실행
4  실행
5  실행
6  실행
7  실행
8  실행
9  실행
10  실행
11  실행


In [6]:
List = data_merge(list_raw, raw_data, '분석용')

merge 시작
key : 1, value : REST
pass
key : 2, value : CBTTF
정답률
과제 완료
key : 3, value : CBTTB
정답률
과제 완료
key : 4, value : GNG
정답률, 반응시간
과제 완료
key : 5, value : TWOBACK
정답률, 반응시간
과제 완료
key : 6, value : STRC
정답률, 반응시간
과제 완료
key : 7, value : STRI
정답률, 반응시간
과제 완료
key : 8, value : VFT
pass
key : 9, value : REPEAT
정답률
과제 완료
key : 10, value : MIND
정답률
과제 완료
key : 11, value : FOLD
정답률
과제 완료
caculate 완료


In [7]:
List.describe()

,age,CBTTF_Cor,CBTTF_ACC,CBTTF_max,CBTTB_Cor,CBTTB_ACC,CBTTB_max,GNG_Cor,GNG_RT,GNG_ACC,...,MIND_ACC,MIND_max,FOLD_Cor,FOLD_ACC,FOLD_max,GNG_z,STRC_z,STRI_z,STRE,STRE_z
count,707.000000,557.000000,557.000000,557.000000,525.000000,525.000000,525.000000,512.000000,512.000000,512.000000,...,449.000000,449.000000,447.000000,447.000000,447.000000,505.000000,414.000000,426.000000,4.590000e+02,389.000000
mean,4.889675,4.332136,0.478371,4.701975,4.925714,0.470715,4.552381,45.275391,620.488498,0.747339,...,0.533632,21.752784,4.892617,0.352622,13.695749,inf,inf,0.666368,-1.066556e+06,-inf
std,5.931072,2.925808,0.324840,1.381095,5.292603,0.301776,1.286905,19.979842,306.422968,0.226367,...,0.280149,3.790719,3.325687,0.238992,1.909747,NaN,NaN,0.252894,4.890182e+05,NaN
min,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,1.000000,0.263123,0.012170,0.011502,-1.824024e+06,-inf
25%,0.000000,1.000000,0.111111,3.000000,2.000000,0.222222,3.000000,28.000000,520.662267,0.635606,...,0.357143,20.000000,2.500000,0.214286,14.000000,1.003696,0.467685,0.563951,-1.381229e+06,-0.050101
50%,3.000000,5.000000,0.555556,5.000000,5.000000,0.555556,5.000000,52.000000,700.825581,0.833333,...,0.600000,24.000000,5.000000,0.357143,14.000000,1.218860,0.670007,0.740872,-1.216148e+06,0.072545
75%,9.000000,7.000000,0.777778,6.000000,7.000000,0.666667,6.000000,60.000000,819.837582,0.906611,...,0.750000,24.000000,7.000000,0.500000,14.000000,1.604876,0.808596,0.835119,-1.023423e+06,0.196911
max,100.000000,9.000000,1.000000,7.000000,60.000000,1.000000,7.000000,75.000000,1382.000000,1.000000,...,1.000000,24.000000,13.000000,1.000000,14.000000,inf,inf,1.326065,1.940000e+03,0.722768


In [14]:
List1 = List[List.age > 6]
List1.describe()

,age,CBTTF_Cor,CBTTF_ACC,CBTTF_max,CBTTB_Cor,CBTTB_ACC,CBTTB_max,GNG_Cor,GNG_RT,GNG_ACC,...,MIND_ACC,MIND_max,FOLD_Cor,FOLD_ACC,FOLD_max,GNG_z,STRC_z,STRI_z,STRE,STRE_z
count,309.000000,301.000000,301.000000,301.000000,297.000000,297.000000,297.000000,294.000000,294.000000,294.000000,...,286.000000,286.000000,281.000000,281.000000,281.0,294.000000,286.000000,279.000000,2.810000e+02,274.000000
mean,9.750809,5.740864,0.636092,5.342193,6.084175,0.597554,5.033670,53.054422,787.561491,0.851302,...,0.635420,21.793706,5.177936,0.369853,14.0,inf,inf,0.746318,-1.292978e+06,-inf
std,6.014542,1.758586,0.196154,0.905086,4.513828,0.195950,0.989238,11.216627,149.481282,0.092766,...,0.190641,2.133138,2.845487,0.203249,0.0,NaN,NaN,0.145556,2.499944e+05,NaN
min,7.000000,0.000000,0.000000,3.000000,0.000000,0.000000,1.000000,11.000000,0.000000,0.333333,...,0.000000,9.000000,0.000000,0.000000,14.0,0.263123,0.014942,0.114550,-1.762421e+06,-inf
25%,8.000000,5.000000,0.555556,5.000000,4.000000,0.444444,5.000000,49.000000,703.781395,0.803092,...,0.500000,20.000000,3.000000,0.214286,14.0,0.960564,0.557909,0.684700,-1.445958e+06,-0.054829
50%,9.000000,6.000000,0.666667,5.000000,6.000000,0.666667,5.000000,55.000000,781.204167,0.864022,...,0.650000,20.000000,5.000000,0.357143,14.0,1.096625,0.704380,0.763465,-1.322412e+06,0.085064
75%,11.000000,7.000000,0.777778,6.000000,7.000000,0.777778,6.000000,60.000000,858.348039,0.909091,...,0.750000,24.000000,7.000000,0.500000,14.0,1.263529,0.817472,0.838512,-1.180806e+06,0.200616
max,100.000000,9.000000,1.000000,7.000000,37.000000,1.000000,7.000000,73.000000,1382.000000,1.000000,...,1.000000,24.000000,13.000000,0.928571,14.0,inf,inf,1.101834,1.468419e+03,0.658687


In [26]:
List2 = List1[List1.age < 14]
for i in List2:
    if 'RT' in i:
        print(i)
        List2[i][List2[i] == 0] = List2[i].mean(skipna=True)
        print(List2[i])
List2.describe()

GNG_RT
0       803.450000
1       734.066667
2       923.200000
3       883.500000
4       811.783784
5       829.846154
6      1109.961538
7       874.593750
8       957.363636
9       772.767442
10      877.468750
11      809.694444
12      941.516129
13      848.156250
14      831.142857
15      697.550000
16      810.378378
17      871.777778
18      669.739130
19      709.463415
20      721.952381
21      735.230769
22      716.590909
23     1047.758621
24      844.972973
25      744.333333
26      919.555556
27      763.105263
28      768.756757
29      785.416667
          ...     
477     756.342105
478     738.000000
479     957.727273
480            NaN
517     716.000000
536     510.431818
539     502.531915
556            NaN
561            NaN
574     791.378402
646    1072.058824
647    1009.611111
649     794.428571
652     955.437500
654            NaN
655     824.277778
657     939.235294
661     703.600000
662     862.263158
674     918.157895
676     817.850000
681  

C:\Users\OBELAB_JH_DESKTOP\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """



STRC_RT
0      1166.644444
1      1160.822222
2      1192.023810
3      1252.300000
4      1245.697674
5      1159.911111
6      1515.869565
7      1273.232558
8      1356.976190
9      1188.600000
10     1307.860465
11     1304.292683
12     1315.325581
13     1207.488889
14     1112.727273
15     1072.116279
16     1296.136364
17     1190.186047
18     1163.441860
19     1279.431818
20     1212.266667
21     1122.590909
22     1148.266667
23     1219.475000
24     1263.377778
25     1255.000000
26     1257.232558
27     1272.800000
28     1136.022222
29     1229.600000
          ...     
477    1418.550000
478    1150.076923
479    1339.424242
480            NaN
517    1535.576923
536     907.577778
539    1004.177778
556            NaN
561            NaN
574    1234.170988
646    1579.727273
647    1462.526316
649    1209.666667
652    1311.523810
654            NaN
655    1680.217391
657    1535.641026
661    1570.920000
662    1551.696970
674    1350.230769
676    1468.418605
681

,age,CBTTF_Cor,CBTTF_ACC,CBTTF_max,CBTTB_Cor,CBTTB_ACC,CBTTB_max,GNG_Cor,GNG_RT,GNG_ACC,...,MIND_ACC,MIND_max,FOLD_Cor,FOLD_ACC,FOLD_max,GNG_z,STRC_z,STRI_z,STRE,STRE_z
count,297.000000,290.000000,290.000000,290.000000,289.000000,289.000000,289.000000,286.000000,286.000000,286.000000,...,279.000000,279.000000,273.000000,273.000000,273.0,286.000000,280.000000,272.000000,2.740000e+02,268.000000
mean,8.976431,5.775862,0.639913,5.358621,6.103806,0.597563,5.031142,52.947552,796.912516,0.851718,...,0.633790,21.792115,5.080586,0.362899,14.0,inf,inf,0.745993,-1.299999e+06,-inf
std,1.615837,1.672525,0.186699,0.877838,4.554792,0.192974,0.962346,10.922087,127.824904,0.088093,...,0.191777,1.992741,2.752156,0.196583,0.0,NaN,NaN,0.139681,2.387066e+05,NaN
min,7.000000,0.000000,0.000000,3.000000,0.000000,0.000000,1.000000,11.000000,283.615385,0.333333,...,0.000000,20.000000,0.000000,0.000000,14.0,0.263123,0.029861,0.114550,-1.762421e+06,-inf
25%,8.000000,5.000000,0.555556,5.000000,4.000000,0.444444,5.000000,49.000000,709.854704,0.803279,...,0.500000,20.000000,3.000000,0.214286,14.0,0.958724,0.556675,0.686168,-1.446053e+06,-0.054611
50%,9.000000,6.000000,0.666667,5.000000,6.000000,0.666667,5.000000,55.000000,783.451229,0.863636,...,0.650000,20.000000,5.000000,0.357143,14.0,1.093128,0.703640,0.763727,-1.324762e+06,0.088033
75%,10.000000,7.000000,0.777778,6.000000,7.000000,0.777778,6.000000,60.000000,860.410590,0.907692,...,0.750000,24.000000,7.000000,0.500000,14.0,1.249224,0.816808,0.835859,-1.187351e+06,0.198146
max,12.000000,9.000000,1.000000,7.000000,37.000000,1.000000,7.000000,73.000000,1382.000000,1.000000,...,1.000000,24.000000,13.000000,0.928571,14.0,inf,inf,1.101834,1.468419e+03,0.658687


In [38]:
path = os.getcwd()
file = os.path.join(path, 'WJ_data.xlsx')
List = pd.read_excel(file, columns = 0, index_col = 0)
List.describe()

,childSex,completed,age,CBTTF_Cor,CBTTF_ACC,CBTTF_max,CBTTB_Cor,CBTTB_ACC,CBTTB_max,GNG_Cor,...,MIND_Cor,MIND_ACC,MIND_max,FOLD_Cor,FOLD_ACC,FOLD_max,GNG_z,STRC_z,STRI_z,STRE_z
count,149.000000,149.0,149.000000,149.000000,149.000000,149.000000,149.000000,149.000000,149.000000,149.000000,...,147.000000,147.000000,147.0,146.000000,146.000000,146.0,149.000000,148.000000,148.000000,148.000000
mean,0.523490,0.0,9.020134,5.946309,0.658816,5.389262,5.624161,0.622521,5.181208,56.671141,...,14.306122,0.715306,20.0,6.273973,0.448141,14.0,1.098425,0.681123,0.824407,0.143284
std,0.501132,0.0,1.686472,1.609873,0.179764,0.859851,1.726097,0.191585,0.900853,7.175826,...,3.358922,0.167946,0.0,2.462184,0.175870,0.0,0.204181,0.165966,0.091065,0.129080
min,0.000000,0.0,7.000000,0.000000,0.000000,3.000000,0.000000,0.000000,3.000000,30.000000,...,0.000000,0.000000,20.0,1.000000,0.071429,14.0,0.546896,0.134731,0.344837,-0.102695
25%,0.000000,0.0,7.000000,5.000000,0.555556,5.000000,5.000000,0.500000,5.000000,52.000000,...,13.000000,0.650000,20.0,4.000000,0.285714,14.0,0.960693,0.598023,0.775137,0.053724
50%,1.000000,0.0,9.000000,6.000000,0.666667,6.000000,6.000000,0.666667,5.000000,57.000000,...,15.000000,0.750000,20.0,6.000000,0.428571,14.0,1.073758,0.698113,0.822319,0.119582
75%,1.000000,0.0,10.000000,7.000000,0.777778,6.000000,7.000000,0.777778,6.000000,62.000000,...,17.000000,0.850000,20.0,7.750000,0.553571,14.0,1.225337,0.794455,0.877845,0.208206
max,1.000000,0.0,12.000000,9.000000,1.000000,7.000000,9.000000,1.000000,7.000000,73.000000,...,20.000000,1.000000,20.0,13.000000,0.928571,14.0,1.577972,1.035065,1.089035,0.646552


In [37]:
for i in List:
    if 'max' in i:
        print(List[i].value_counts())

6    67
5    53
4    17
7     8
3     4
Name: CBTTF_max, dtype: int64
5    66
6    46
4    23
7     8
3     6
Name: CBTTB_max, dtype: int64
9     35
10    31
11    22
8     21
12    14
13     9
7      6
14     5
15     3
6      2
4      1
Name: GNG_max, dtype: int64
1.0    60
2.0    37
3.0    18
5.0    12
6.0    10
4.0     9
Name: REPEAT_max, dtype: int64
20.0    147
Name: MIND_max, dtype: int64
14.0    146
Name: FOLD_max, dtype: int64


In [39]:
s_data = data_scoring(raw, '분석용scoring')

scoring data listing 시작
key : 1, value : REST
key : 2, value : CBTTF
CBTTF 과제 완료
key : 3, value : CBTTB
CBTTB 과제 완료
key : 4, value : GNG


IndexError: list index out of range

In [ ]:
figure_out_perf(s_data, gtype = 'bar', level = True)

In [ ]:
figure_out_perf(s_data, gtype = 'image')